# Quick Start: Polymer Field Theory in 5 Minutes

This tutorial provides a rapid introduction to `polymerfts` for polymer field theory simulations.

**What you'll learn:**
- How to compute chain propagators
- How to calculate polymer concentrations
- Basic visualization of results

**Prerequisites:** Basic Python and NumPy knowledge. Basic polymer physics experience required.

## 1. Setup

First, import the necessary libraries:

In [ ]:
import os
os.environ["OMP_MAX_ACTIVE_LEVELS"] = "1"  # Optimize threading

import numpy as np
import matplotlib.pyplot as plt
from polymerfts import PropagatorSolver

print("polymerfts imported successfully!")

## 2. Your First Simulation: A Simple Homopolymer

Let's simulate a homopolymer (a chain made of identical monomers) in a 1D domain.

### Step 1: Create a Solver

The `PropagatorSolver` handles all the complex setup for you:

In [ ]:
# Create a 1D solver
solver = PropagatorSolver(
    nx=[64],        # 64 grid points
    lx=[4.0],       # Domain length = 4.0 (in units of polymer size)
    ds=0.01,        # Contour step size
    bond_lengths={"A": 1.0}  # Monomer type "A" with bond length 1.0
)

# Add a homopolymer chain
# Format: [monomer_type, length, start_node, end_node]
solver.add_polymer(
    volume_fraction=1.0,
    blocks=[["A", 1.0, 0, 1]]  # Single block of type A, length 1.0
)

print(solver.info)

### Step 2: Set the Potential Field

The potential field $w(r)$ affects how the polymer distributes in space.
- $w = 0$: Free diffusion (uniform distribution)
- $w > 0$: Repulsive region (polymer avoids)
- $w < 0$: Attractive region (polymer prefers)

In [ ]:
# Create a simple potential: attractive in the center, repulsive at edges
x = np.linspace(0, 4.0, 64)
w_A = 5.0 * (x - 2.0)**2 - 5.0  # Parabolic well centered at x=2

# Set the field
solver.set_fields({"A": w_A})

# Plot the potential
plt.figure(figsize=(8, 3))
plt.plot(x, w_A, 'b-', linewidth=2)
plt.xlabel('Position x')
plt.ylabel('Potential w(x)')
plt.title('External Potential Field')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.show()

### Step 3: Compute and Visualize Propagator Evolution

The **propagator** $q(r, s)$ represents the probability of finding a chain segment at position $r$ after evolving along the chain contour for "time" $s$.

Starting from a uniform distribution, let's watch it evolve:

In [ ]:
# Start with uniform initial condition
q = np.ones(64)

# Store snapshots
n_steps = 100  # Total chain length = n_steps * ds = 1.0
snapshots = [q.copy()]
snapshot_times = [0]

# Evolve the propagator
for step in range(n_steps):
    q = solver.advance(q, "A")
    if (step + 1) % 20 == 0:
        snapshots.append(q.copy())
        snapshot_times.append((step + 1) * 0.01)

# Plot the evolution
plt.figure(figsize=(10, 4))
colors = plt.cm.viridis(np.linspace(0, 1, len(snapshots)))
for i, (snap, t) in enumerate(zip(snapshots, snapshot_times)):
    plt.plot(x, snap, color=colors[i], label=f's = {t:.2f}', linewidth=2)

plt.xlabel('Position x')
plt.ylabel('Propagator q(x, s)')
plt.title('Propagator Evolution Along Chain Contour')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

print("The propagator concentrates where the potential is attractive (negative).")

## 3. AB Diblock Copolymer in 2D

Now let's simulate a more interesting system: an **AB diblock copolymer** (two different block types joined together) in 2D.

In [ ]:
# Create a 2D solver for AB diblock
solver_2d = PropagatorSolver(
    nx=[32, 32],           # 32x32 grid
    lx=[4.0, 4.0],         # 4x4 domain
    ds=0.01,
    bond_lengths={"A": 1.0, "B": 1.0}
)

# Add AB diblock: A block (50%) connected to B block (50%)
# Block format: [type, length, start_node, end_node]
solver_2d.add_polymer(
    volume_fraction=1.0,
    blocks=[
        ["A", 0.5, 0, 1],  # A block from node 0 to node 1
        ["B", 0.5, 1, 2]   # B block from node 1 to node 2
    ]
)

print(solver_2d.info)

In [ ]:
# Create striped potential fields (mimics phase separation)
x = np.linspace(0, 4.0, 32)
y = np.linspace(0, 4.0, 32)
X, Y = np.meshgrid(x, y, indexing='ij')

# A prefers left side, B prefers right side
w_A = 2.0 * np.sin(2 * np.pi * X / 4.0).flatten()
w_B = -w_A  # Opposite preference

solver_2d.set_fields({"A": w_A, "B": w_B})

# Visualize the potentials
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

im0 = axes[0].imshow(w_A.reshape(32, 32).T, origin='lower', 
                     extent=[0, 4, 0, 4], cmap='RdBu_r')
axes[0].set_title('Potential for A monomers')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(w_B.reshape(32, 32).T, origin='lower',
                     extent=[0, 4, 0, 4], cmap='RdBu_r')
axes[1].set_title('Potential for B monomers')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Evolve propagator for the full chain
q = np.ones(32 * 32)  # Start uniform

# A block: 50 steps
for _ in range(50):
    q = solver_2d.advance(q, "A")

# B block: 50 steps  
for _ in range(50):
    q = solver_2d.advance(q, "B")

# Visualize final propagator
plt.figure(figsize=(6, 5))
plt.imshow(q.reshape(32, 32).T, origin='lower', 
           extent=[0, 4, 0, 4], cmap='hot')
plt.colorbar(label='q(r, s=1)')
plt.title('Final Propagator for AB Diblock')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

print("The propagator shows where the full chain (A-B) is likely to be found.")

## 4. What's Next?

You've learned the basics! Here's where to go next:

### Recommended Learning Order

1. **`NonInteractingChainsInPotentialFields/Diblock.ipynb`** - Detailed AB diblock with concentrations and animations
2. **`NonInteractingChainsInPotentialFields/NonPeriodicBC.ipynb`** - Confined polymers with reflecting/absorbing boundaries
3. **`PolymerFieldTheory/`** - Mathematical foundations of field theory
4. **`SelfConsistentFieldTheory/NaiveSCFT.ipynb`** - Finding equilibrium polymer phases
5. **`SelfConsistentFieldTheory/Cylinder.ipynb`** - Production SCFT with box optimization
6. **`LangevinFieldTheoreticSimulation/`** - Fluctuating field simulations

### Key Concepts to Explore
- **Branched polymers**: Star, comb, and bottle-brush architectures
- **Polymer mixtures**: Multiple polymer species
- **Self-consistent field theory (SCFT)**: Finding equilibrium morphologies
- **Langevin FTS**: Including compositional fluctuations

## Quick Reference

### PropagatorSolver Parameters

| Parameter | Description | Example |
|-----------|-------------|--------|
| `nx` | Grid points per dimension | `[64]`, `[32, 32]`, `[16, 16, 16]` |
| `lx` | Box size per dimension | `[4.0]`, `[4.0, 4.0]` |
| `bc` | Boundary conditions | `["periodic", "periodic"]`, `["reflecting", "absorbing"]` |
| `ds` | Contour step size | `0.01` (typical) |
| `bond_lengths` | Statistical segment lengths | `{"A": 1.0, "B": 1.2}` |
| `method` | Numerical method | `"pseudospectral"` (default), `"realspace"` |
| `platform` | Computation platform | `"auto"`, `"cpu-mkl"`, `"cuda"` |

### Common Polymer Architectures

```python
# Homopolymer
blocks = [["A", 1.0, 0, 1]]

# AB Diblock
blocks = [["A", 0.5, 0, 1], ["B", 0.5, 1, 2]]

# ABC Triblock
blocks = [["A", 0.3, 0, 1], ["B", 0.4, 1, 2], ["C", 0.3, 2, 3]]

# 3-arm star (A-B-A meeting at center node 0)
blocks = [["A", 0.33, 0, 1], ["B", 0.33, 0, 2], ["A", 0.33, 0, 3]]
```